In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from natsort import natsorted

In [39]:
files = ['acc_climbingup_forearm.csv',
         'acc_climbingup_waist.csv',
         'acc_walking_forearm.csv',
         'acc_walking_waist.csv']
s = [None] * len(files)
for i, path in enumerate(files):
    data = pd.read_csv(path)
    s[i] = data['attr_x'].to_numpy()
    s[i] = s[i][:26938]
    s[i] = s[i] - s[i].mean()


In [41]:
fig = go.Figure()
for i, x in enumerate(s):
    fig.add_trace(go.Scatter(y=x, mode="lines", showlegend=True, name=files[i]))
fig.show()

In [4]:
def picotar(x, J):
    N = x.shape[0]
    X = np.zeros((N-J, J))
    for i in range(N-J):
        X[i,:] = x[i:(i+J)]
    return X

In [5]:
def pcabase(X):
    M = np.ones((X.shape[0],1)).dot(X.mean(axis=0).reshape(1,X.shape[1]))
    R = (X-M).transpose().dot(X-M)
    l, vet = np.linalg.eig(R)
    inds = np.argsort(l)
    vet = vet[:,inds][:,::-1]
    return l, vet

In [44]:
J = 150
X = picotar(s[0], J)
l, vet = pcabase(X)

In [45]:
fig = px.line(vet[:,:5], range_y=[-1/5,1/5])
fig.show()
fig = px.scatter(l.cumsum()/l.sum(), range_y=[0,1.1])
fig.show()

In [46]:
d = 30
W = vet[:,:d]

In [49]:
i = 1000
X = picotar(s[2], J)
x = X[i,:]
S = x.dot(W)
xh = S.dot(W.transpose())
fig = px.line(xh)
fig.add_trace(go.Scatter(y=x, mode='lines', showlegend=True))

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            # 1st hidden layer
            nn.Flatten(start_dim=1),
            nn.Linear(300, 100),
            nn.Tanh(),

            # 2nd hidden layer
            nn.Linear(100, 2)
            # nn.Softmax(dim=1),

#             # output layer
#             nn.Linear(15, num_classes),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits

In [ ]:
def ajusteDados(X):
    N = X.shape[0]
    Nj = X.shape[2]
    S = np.zeros((N*(Nj-151), 2, 150))
    y = np.zeros((N*(Nj-151), 2))
    for i in range(N):
        for j in range(Nj-151):
            S[(i*(Nj-151)+j),:,:] = X[i,:,j:(j+150)]
            y[(i*(Nj-151)+j),:] = X[i,:,(j+150)]
    return S, y

In [ ]:
def predlin(dados, ordem):
    N = dados.shape[0]
    X = dados[:,:1200].reshape(N,2,600) / 50   # Normalizando para a faixa de -1 a 1 aprox.
    y = dados[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
    
    X_train, s_train = ajusteDados(X_train)
    X_val, s_val = ajusteDados(X_val)
    X_test, s_test = ajusteDados(X_test)
    
    aux = np.linalg.inv(np.matmul(X_train[:,0,-ordem:].transpose(), X_train[:,0,-ordem:]))
    w1 = np.matmul(np.matmul(aux, X_train[:,0,-ordem:].transpose()), s_train[:,0])
    
    aux = np.linalg.inv(np.matmul(X_train[:,1,-ordem:].transpose(), X_train[:,1,-ordem:]))
    w2 = np.matmul(np.matmul(aux, X_train[:,1,-ordem:].transpose()), s_train[:,1])

    return w1, w2

In [ ]:
ordem = 12
w1, w2 = predlin(D[3], ordem)